# Homework Week 1

You will find data, in 3 csv files, in the "extracts" folder. 

Your job is to create a SQL database, in Postgres, to store this data in _normalized_ form (3NF!). You will submit: 

1. A .sql file containing sql queries to create the database and tables. 
2. A .py file containing Python code to load the data from the csv's into the database. 

In [33]:
import pandas as pd
import psycopg2
import numbers
import math
import ast

In [34]:
class db_connection():
    def __init__(self,
                 dbname="company_db",
                 user="postgres",
                 host="localhost"):
        self.configure_conn(dbname, user, host)
        self.connect()
        self.open_cursor()
    
    def __del__(self):
        if self.cur is not None:
            self.close_cursor()
        if self.conn is not None:
            self.disconnect()

    def connect(self):
        conn_str = f"dbname={self.dbname} user={self.user} host={self.host}"
        self.conn = psycopg2.connect(conn_str)
    
    def configure_conn(self,
                        dbname,
                        user,
                        host):
        self.dbname=dbname
        self.user=user
        self.host=host
        
    def disconnect(self, close_cursor_b=False):
        if close_cursor_b:
            self.close_cursor
        self.conn.close()
        self.conn = None
        
    def is_connected(self):
        if self.conn is None:
            return False
        else:
            return True
        
    def open_cursor(self):
        self.cur = self.conn.cursor()
        
    def close_cursor(self):
        self.cur.close()
        self.cur = None

    def select_query(self, query):
        self.cur.execute(query)
        return self.cur.fetchall()
        
    def insert_query(self, query, var_tuple):
        self.cur.execute(query, var_tuple)
        self.conn.commit()
        
    def _delete_query(self, query):
        # Not a fan of providing direct delete methods, testing only...
        self.cur.execute(query)
        self.conn.commit()

In [35]:
def build_query(row, table, query_type):
    # Abstracted to allow for additon of UPDATE and SELECT later on
    if query_type == "INSERT":
        return build_insert_query(row, table)
    else:
        return ""

def build_insert_query(row, table):#, types_dict):
    query_columns = ""
    variable_list = list()
    for key in row.keys():
        # Insert is columns then values, so construct as two strings
        query_columns += f"{key}, "
        # Use format value to handle value types in SQL
        variable_list.append(row[key])
    values = "%s, " * len(row.keys())
    # Index to -2 to remove ", " from last value
    query = f"INSERT INTO {table} ({query_columns[:-2]}) VALUES ({values[:-2]})"
    return query, variable_list

def load_table(table_name, frame):
    # Build table
    for index, row in frame.iterrows():
        query_str, var_tuple = build_query(row, table_name, "INSERT")
        #print(query_str)
        conn.insert_query(query_str, var_tuple)
        
def _torch_tables(conn, tables=['products', 'orders', 'customers', 'employees', 'offices',  ]):
    for table in tables:
        conn._delete_query(f"DELETE FROM {table};") # This is for testing only, method should not be used.


In [36]:
conn = db_connection()


In [37]:
# Load files
products = pd.read_csv('extracts/products.csv') 
orders_full = pd.read_csv('extracts/orders.csv')
employees_full = pd.read_csv('extracts/employees.csv')

# Convert byte columns to string literals
orders_full["customer_location"] = orders_full["customer_location"].apply(ast.literal_eval)
employees_full["office_location"] = employees_full["office_location"].apply(ast.literal_eval)

# Fix nulls to None as it messes with the times
orders_full = orders_full.where(pd.notnull(orders_full), None)


In [38]:
# From products, take all columns
load_table('products', products)

# Split the employees table into two tables; offices and employees
offices_col_list = {'office_code', 'city', 'state', 'country', 
                    'office_location' }
offices_df = employees_full[offices_col_list].drop_duplicates()
load_table('offices', offices_df)

# Second part, employees
employees_col_list = {'employee_number', 'last_name', 'first_name',
                      'reports_to', 'job_title'}
employees_df = employees_full[employees_col_list]
load_table('employees', employees_df)

# Split orders into three tables; customers, orders and order_items
customers_col_list = ['customer_number', 'customer_name',
                      'contact_last_name', 'contact_first_name',
                      'city', 'state', 'country',
                      'sales_rep_employee_number', 'credit_limit',
                      'customer_location']
customers_df = orders_full[customers_col_list].drop_duplicates()
load_table('customers', customers_df)

#Second part, orders
orders_col_list = [ 'order_number', 'customer_number', 'order_date',
                   'required_date', 'shipped_date', 'status',
                   'comments' ]
orders_df = orders_full[orders_col_list].drop_duplicates()
load_table('orders', orders_df)

# Third part, order_items

# Create new index for order_item that combines the order_number with the line number
orders_full["order_item_number"] = orders_full["order_number"].astype(str) + "-" +orders_full["order_line_number"].astype(str)
order_items_col_list = [ 'order_item_number', 'order_number',
                        'product_code', 'quantity_ordered',
                        'price_each', 'order_line_number']
order_items_df = orders_full[order_items_col_list]
load_table('order_items', order_items_df)


In [39]:
if conn.is_connected():
    conn.disconnect(True)